In [30]:
import os

import pandas as pd

folder_path = r"C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\초장대 K-지하고속도로 인프라 안전 및 효율 향상 기술 개발(2차년도)\승용차추종행태_원본데이터\3차로_점선\1750"
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]
print("mer_list : ", mer_list)
for i in range(len(mer_list)):
    number = [5, 30]
    file_path = os.path.join(folder_path, mer_list[i])
    # 파일 읽기
    with open(file_path, "r", encoding="utf-8", errors="ignore") as file:
        lines = file.readlines()
    # 데이터가 시작하는 인덱스 찾기
    data_start_idx = None

    for j, line in enumerate(lines):
        if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
            data_start_idx = j
            break

    # 데이터프레임 생성
    if data_start_idx is not None:
        # 컬럼명 추출
        columns = lines[data_start_idx].strip().split(";")

        # 데이터 부분 추출 및 가공
        data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
        data = [line.strip().split(";") for line in data_lines if line.strip()]

        # 데이터프레임 생성
        df = pd.DataFrame(data, columns=columns)
        # 컬럼명을 정리 (공백 제거)
        df.columns = df.columns.str.strip()

        # Measurem., t(Entry), Vehicle type 컬럼의 데이터에서 공백 제거후 정수형 변환
        cols = ["Measurem.", "t(Entry)", "Vehicle type", "v[km/h]"]
        for col in cols:
            df[col] = pd.to_numeric(df[col].astype(str).str.strip())
    ###############################################################################################
        #measure_list = [20010, 20060, 20110]
        measure_list = [30010, 30060, 30110]
        result_list = []
        for measure in measure_list:
            print("measure : ", measure)
            df_filtered = df[(df["t(Entry)"] != -1.00) & (df["Measurem."] == measure)].reset_index(drop=True)
            heavy_list = df_filtered.index[df_filtered["Vehicle type"]>= 600].tolist()
            merged_list = []
            for idx in range(len(heavy_list)-1):
                start_num = heavy_list[idx]
                df_new = df_filtered[start_num : heavy_list[idx+1]].copy()
                df_new = df_new[(df_new["Vehicle type"] != 300) & (df_new["Vehicle type"] <600)].reset_index(drop=True)
                if df_new.empty:
                    continue
                row_count = len(df_new)
                v_avg = df_new["v[km/h]"].mean()
                merged_list.append({"차량수":row_count, "평균속도":v_avg})

            merged_df = pd.DataFrame(merged_list)

            grouped_df = merged_df.groupby("차량수", as_index=False).agg(
                평균속도 = ("평균속도", "mean"),
                그룹수 = ("평균속도", "count")
            )
            result_list.append(grouped_df)
        final_df = pd.concat(result_list, axis=1)
        final_df.columns = ['진입부_차량수', '진입부_평균속도', '진입부_그룹수','본선부_차량수', '본선부_평균속도', '본선부_그룹수', '진출부_차량수','진출부_평균속도', '진출부_그룹수']

        excel_path = os.path.join(folder_path, f"3차로_3차선_점선_1750_{number[i]}%.xlsx")
        final_df.to_excel(excel_path, index=False)
        print("작업완료 : ", excel_path)

mer_list :  ['network03_수정_241.mer', 'network03_수정_351.mer']
measure :  30010
measure :  30060
measure :  30110
작업완료 :  C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\초장대 K-지하고속도로 인프라 안전 및 효율 향상 기술 개발(2차년도)\승용차추종행태_원본데이터\3차로_점선\1750\3차로_3차선_점선_1750_5%.xlsx
measure :  30010
measure :  30060
measure :  30110
작업완료 :  C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\초장대 K-지하고속도로 인프라 안전 및 효율 향상 기술 개발(2차년도)\승용차추종행태_원본데이터\3차로_점선\1750\3차로_3차선_점선_1750_30%.xlsx
